In [20]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [21]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [22]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [23]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [245]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_layers(self, layers):
            self.layers = layers
        
        def set_weights(self, weights_and_biases):
            self.w, self.b = [], []

            for i, weight_bias in enumerate(weights_and_biases):
                self.w += [weight_bias[0]]
                self.b += [weight_bias[1]]
        
        def set_mutation_sigma(self, n):
            #self.mutation_sigma = 3 - (2 - 40 / (10 + n))
            self.mutation_sigma = 2.5 - (3 - 1200 / (200 + n))
        
        def predict(self, input):
            forward_pass = input.T

            for i in range(0, len(self.layers) - 1):
                if i < len(self.layers) - 2:
                    forward_pass = sigmoid(self.w[i].T @ forward_pass + self.b[i])
                else:
                    forward_pass = self.w[i].T @ forward_pass + self.b[i]

            return forward_pass.reshape(-1)

        
        def get_layer_shapes(self):
            weights, biases = [], []

            for w, b in zip(self.w, self.b):
                weights += [w.shape]
                biases += [b.shape]

            return weights, biases
        
        def __add__(self, other):
            w, b = [], []

            for i in range(len(self.layers) - 1):
                w += [(self.w[i] + other.w[i]) / 2 + np.random.normal(0, self.mutation_sigma, (self.layers[i], self.layers[i + 1]))]
            
            for i in range(len(self.layers) - 1):
                b += [(self.b[i] + other.b[i]) / 2 + np.random.normal(0, self.mutation_sigma, (self.layers[i + 1], 1))]

            return zip(w, b)

    
    def __init__(self, n = 100, hidden_layers = False, random_state = None, verbose = 0):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        if not hidden_layers:
            self.layers = [1]
        else:
            self.layers = hidden_layers + [1]
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)

        self.verbose = verbose
    
    def fit(self, X_train, y_train, epochs = 100):
        self.layers = [X_train.shape[1]] + self.layers

        for key in self.nets.keys():
            self.nets[key].set_layers(self.layers)
            
            w, b = [], []

            for i in range(len(self.layers) - 1):
                w += [np.random.uniform(-3, 3, (self.layers[i], self.layers[i + 1]))]
            
            for i in range(len(self.layers) - 1):
                b += [np.random.uniform(-3, 3, (self.layers[i + 1], 1))]
                    
            self.nets[key].set_weights(zip(w, b))

        for epoch in range(epochs):
            if self.verbose == 1:
                print(f"Epoch {epoch}")

            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                pass

    def predict(self, X):
        return self.nets[self.best_net].predict(X)

In [251]:
regressor = ERegressor(n = 100, hidden_layers = [6], random_state = 42, verbose = 0)
regressor.fit(scaled_X_train, y_train, epochs = 1000)

Epoch 0: MAE: 16.501037965868054
Epoch 1: MAE: 4.481795910353849
Epoch 12: MAE: 4.181537268461246
Epoch 18: MAE: 4.0025040304028625
Epoch 21: MAE: 3.790619909594943
Epoch 31: MAE: 3.321072565234875
Epoch 36: MAE: 3.1402337529725437
Epoch 157: MAE: 2.9937673821255975
Epoch 160: MAE: 2.9107149443223634
Epoch 183: MAE: 2.593807625969956
Epoch 241: MAE: 2.542751375975385
Epoch 267: MAE: 2.4764925681887933
Epoch 301: MAE: 2.4311760375427682
Epoch 315: MAE: 2.4259625886766787
Epoch 350: MAE: 2.304664013195951
Epoch 426: MAE: 2.226990605002127
Epoch 492: MAE: 2.2216131198451348
Epoch 494: MAE: 2.2193141571368633
Epoch 496: MAE: 2.2035666168828016
Epoch 515: MAE: 2.1990959533988237
Epoch 521: MAE: 2.1924314210811446
Epoch 545: MAE: 2.1179703454284224
Epoch 635: MAE: 2.0914126327442406
Epoch 643: MAE: 2.088403794514863
Epoch 645: MAE: 2.0855598265900723
Epoch 646: MAE: 2.068472567716153
Epoch 667: MAE: 1.9898258786529126
Epoch 807: MAE: 1.9780567973219205
Epoch 852: MAE: 1.977777016166707
Epoch

In [229]:
regressor.nets[regressor.best_net].get_layer_shapes()

([(9, 1)], [(1, 1)])

In [193]:
regressor.nets[0].w[0]

array([[-0.75275929,  2.70428584,  1.39196365,  0.59195091, -2.06388816,
        -2.06403288, -2.65149833,  2.19705687,  0.60669007],
       [ 1.24843547, -2.87649303,  2.81945911,  1.99465584, -1.72596534,
        -1.9090502 , -1.89957294, -1.17454654,  0.14853859],
       [-0.40832989, -1.25262516,  0.67111737, -2.16303684, -1.24713211,
        -0.80182894, -0.26358009,  1.71105577, -1.80195731],
       [ 0.08540663,  0.55448741, -2.72129752,  0.64526911, -1.97685526,
        -2.60969044,  2.69331322,  2.7937922 ,  1.85038409],
       [-1.17231738, -2.41396732,  1.10539816, -0.35908504, -2.26777059,
        -0.02893854, -2.79366887,  2.45592241, -1.44732011],
       [ 0.97513371, -1.12973354,  0.12040813,  0.28026168, -1.89087327,
         2.81750777,  1.65079694,  2.63699365,  2.3689641 ],
       [ 0.58739987,  2.53124541, -2.46904499, -1.82410283, -2.72863627,
        -1.04801802, -0.66793626, -1.37190581,  1.97242505],
       [-0.85948004, -1.31439294,  0.2561765 , -2.15445465,  1

In [174]:
scaled_X_train.T.shape

(9, 318)

In [175]:
regressor.nets[1].get_layer_shapes()

([(9, 9), (9, 1)], [(9, 1), (1, 1)])

In [252]:
y_pred = regressor.predict(scaled_X_test)

In [253]:
mean_absolute_error(y_test, y_pred)

2.038796192889728